## How to make your testing life easier by using unittest.**mock** package

### A small introduction to mocking, APIs and cats

## What is mocking?

>`unittest.mock` is a library for testing in Python. It allows you to replace parts of your system under test with mock objects and make assertions about how they have been used.

![SO](http://localhost:5500/lib/img3.png)

In [1]:
# Hidden imports go here
import sys
import ipytest
import requests
import pyspark
import unittest
from unittest import mock

Let's start with the *very* basics:

In [2]:
def a_random_function(arg1: str, arg2: str) -> str:
    return arg2 + arg1

a_random_function("AMRO", "ABN")

'ABNAMRO'

Now let's now make things slightly less linear:

In [3]:
def another_function(arg1: int, arg2: int, arg3: str, arg4: str) -> str:
    return a_random_function(arg3, arg4) + str((arg1 + arg2)//2)

another_function(2, 4, "ʘ=)∫", "(=ʘᆽ")

'(=ʘᆽʘ=)∫3'

### How is this related to unit testing?    
Let's take the definition of *unit testing*. According to Wikipedia (the main source of information for our century), we can define unit testing as:
>"A software testing method by which individual units of source code — sets of one or more computer program modules together with associated control data, usage procedures, and operating procedures — are tested to determine whether they are fit for use."

The dilemma is:
> " How can we test something over which we have no control? "

In our specific case, let's take `a_random_function(arg1, arg2)` as given.    
Let's say we import it from some other package, that doesn't belong to us.    

We run into this issue many times: the most common time is when we interact with the filesystem.

Let's redefine our `another_function()` as follows:

In [32]:
def another_function(arg1: str, arg2: str) -> str:
    return a_random_function(sys.argv[0], sys.argv[1]) + " - " + str((arg1 + arg2)//2)

another_function(10, 200)

'-f/home/gian/.pyenv/versions/3.7.0/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py - 105'

As you can see, I have no possible control over `sys.argv`: they get defined at runtime, once I run my application.    
How can we account for them, or get *some* degree of control?

Fortunately, `unittest.mock` allows us to perform this task in a relatively relaxed way.

In [28]:
@mock.patch("sys.argv", ["One", "Two"])
def another_function(arg1: str, arg2: str) -> str:
    return a_random_function(sys.argv[0], sys.argv[1]) + " - " + str((arg1 + arg2)//2)

another_function(10, 200)

{'__wrapped__': <function __main__.another_function(arg1: str, arg2: str) -> str>,
 'patchings': [<unittest.mock._patch at 0x7f5818833588>]}

In [ ]:
print(sys.argv)

![cat](http://localhost:5500/lib/img1.jpg)

In [ ]:
# Test-related imports go here

ipytest.autoconfig()
test_args = ["--showlocals", 
            "-x", 
            "--cov-report", 
            "term-missing",
            "--cov",
            "neon.functions"]

## Let's get to our cats 🐈
Since I really love cats, I'd like to know more about them.    
Fortunately, someone created the **completely free** [Cat facts API](https://catfact.ninja/) which can return nice (and interesting) facts about our feline friends.

In [ ]:
requests.get(url="https://catfact.ninja/fact").json()["fact"]

For the purpose of this presentation, I made a **whole application** to collect facts about cats. The application does two main things:
- Collects a certain number of facts by querying the API
- Saves everything into my local Spark cluster, into a well organized table   

I called the application **Neon** (just because that's the first name I got from a random generator)    
The application code and its tests are freely available on my github profile (link at the end of the presentation)

### Let's start making use of it then!

In [45]:
from neon.functions import *

![outline](http://localhost:5500/lib/img4.png)

In [43]:
interesting_facts = process_data(usernumber=3)

2021-08-20 11:41:02,415 - DEBUG - Now arranging API call to https://catfact.ninja/fact
2021-08-20 11:41:09,425 - DEBUG - Time elapsed: 7.0 seconds
2021-08-20 11:41:09,427 - DEBUG - Starting call now
2021-08-20 11:41:09,434 - DEBUG - Starting new HTTPS connection (1): catfact.ninja:443
2021-08-20 11:41:10,054 - DEBUG - https://catfact.ninja:443 "GET /fact HTTP/1.1" 200 None
2021-08-20 11:41:10,060 - DEBUG - Load 1 of 3 is done
2021-08-20 11:41:10,060 - DEBUG - Now arranging API call to https://catfact.ninja/fact
2021-08-20 11:41:14,065 - DEBUG - Time elapsed: 4.0 seconds
2021-08-20 11:41:14,067 - DEBUG - Starting call now
2021-08-20 11:41:14,071 - DEBUG - Starting new HTTPS connection (1): catfact.ninja:443
2021-08-20 11:41:14,641 - DEBUG - https://catfact.ninja:443 "GET /fact HTTP/1.1" 200 None
2021-08-20 11:41:14,645 - DEBUG - Load 2 of 3 is done
2021-08-20 11:41:14,646 - DEBUG - Now arranging API call to https://catfact.ninja/fact
2021-08-20 11:41:18,651 - DEBUG - Time elapsed: 4.0 s

 Each API call generates a JSONs. `process_data()` conveniently packs them into a list, over which we can easily iterate: 

In [ ]:
for entry in interesting_facts:
    print(entry["fact"])

Let's now store these important facts inside my table, so then they don't get lost:

In [ ]:
spark = establish_spark()
group_and_save(spark, facts=interesting_facts)
df = spark.read.table("default.random_cats_facts")

In [ ]:
df.show(100)

As you can see, the whole procedure is very slow. This is due to a random `waiting` variable that I added in order to simulate a slow connection, a very server side, or anything that can get in between.    
Since the `waiting` variable polls a random amount of seconds between 1 and 10, we're talking here about an average waiting time of 5 seconds (if you don't believe me, check the [central limit theorem](https://en.wikipedia.org/wiki/Central_limit_theorem) for statistical reference).    

### What can I do to make the whole thing more efficient, while testing for the main functionality?

There are two main issues over here:
1) I don't wanna wait 5 seconds for each call to the API, but I'd like to be sure that the call works nonetheless (i.e. no test data, I don't care about the data: I want my functionality to work)    
2) I don't wanna save the data I retrieve every single time, since I have no control over it will be saved: I just know that, as soon as I run the application, the data will be saved somewhere. That's not good for testing purposes!

Let's use our mocks:

In [ ]:
# Mention again that this is our stuff
class TestAPI(unittest.TestCase):
    @mock.patch("neon.functions.make_request", return_value=None)
    def test_retrieve_data_without_waiting(self, patched_request):
        t0 = time.perf_counter()
        actual: dict = retrieve_data(waiting=5)
        actual_keys: list = [key for key in actual]
        expected_keys: list = ["fact", "length"]
        t1 = time.perf_counter() - t0
        self.assertEqual(actual_keys, expected_keys)
        self.assertLess(t1, 10)
    
    @mock.patch("neon.functions.retrieve_data", return_value={"fact" : "This is a random fact", "length" : "-99"})
    def test_process_data_with_custom_load(self, patched_retrieve):
        t0 = time.perf_counter()
        actual: list = process_data(usernumber=5, waiting=1)
        expected: list = [{"fact" : "This is a random fact", "length" : "-99"}]*5
        t1 = time.perf_counter() - t0
        self.assertEqual(actual, expected)
        self.assertLess(t1, 10)

In [ ]:
first_test_suite = TestAPI()
ipytest.run(*test_args)

In [ ]:
class TestSparkFunctions(unittest.TestCase):
    @mock.patch("pyspark.sql.readwriter.DataFrameWriter.saveAsTable", new_callable=logging.debug("Dataframe saved!"))
    def test_group_and_save_with_random_load(self):
        pass
    
    @mock.patch("neon.functions.make_request", return_value=None)
    @mock.patch("pyspark.sql.readwriter.DataFrameWriter.saveAsTable", new_callable=logging.debug("Dataframe saved!"))
    def test_group_and_save_with_api_load(self):
        pass

In [ ]:
second_test_suite = TestSparkFunctions()
ipytest.run(*test_args)

### Problem solved!

![cat](http://localhost:5500/lib/img2.jpg)

### Useful links:
[Catfacts (API reference)](https://catfact.ninja/)    
[This application (Git repo)](https://github.com/jean-n92/pytest-mock-presentation)    
[Where to patch (Documentation)](https://docs.python.org/3/library/unittest.mock.html#where-to-patch)    
[Quick start with mock (Documentation)](https://docs.python.org/3/library/unittest.mock.html#quick-guide)
[Difference between mock and stub (StackOverflow)](https://stackoverflow.com/questions/3459287/whats-the-difference-between-a-mock-stub)